# The non-linear Kirchhoff model

## Model and energy

## The isometry constraint

## Discretisation

## Linearisation of the nodal isometry constraint

> For this, it is important ot realize that for the empleyed finite element space $W_h$, the nodal values of the discrete defformation ($u_h(z) : z \in \mathcal{N}_h$) are mutually independent variables in the minimization problem.

## Discrete $H^2$ gradient flow

# Implementation


Besides the work already done for [the linear case](linear-kirchhoff.ipynb), we need to compute the matrix $B_{k-1}$ which enforces the nodal isometry constraint for the solution $d_t u^k$ at step $k$.

The system to solve is (Bartels, 2013):

$$\left(\begin{array}{cc}
  (1 + \alpha \tau) M^{\top} A^{(2)} M & B^{\top}_{k - 1}\\
  B_{k - 1} & 0
\end{array}\right)  \left(\begin{array}{c}
  d_t U^k\\
  \Lambda
\end{array}\right) = \left(\begin{array}{c}
  - \alpha M^{\top} A^{(2)} MU^{k - 1} + \tau F\\
  0
\end{array}\right)$$

The local tensor matrix requires then the discrete gradient matrix $M$, the local tensor for the form $(\nabla u, \nabla v)$ in $P_2$ and the constraints matrix $B_k-1$. Note also that we now have vector valued functions and this will probably require tweaking some of the previous code.